In [1]:
from transformers import pipeline

# Crear el pipeline de question-answering
nlp = pipeline(
    'question-answering', 
    model='mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
    tokenizer=(
        'mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',  
        {"use_fast": False}
    )
)

# Definir la pregunta y el contexto
question = '¿Cual es la película favorita de Manuel Romero?'
context = ('Manuel Romero está colaborando activamente con huggingface/transformers '
           'para traer el poder de las últimas técnicas de procesamiento de lenguaje natural al idioma español'
           'La película favorita de Manuel Romero es Spiderman')

# Ejecutar el pipeline con la pregunta y el contexto
result = nlp(
    {
        'question': question,
        'context': context
    }
)

# Mostrar el resultado
print(f"Respuesta: {result['answer']}")
print(f"Score: {result['score']}")

d:\Cursos\Curso BOOTCAMP IA GENERATIVA-SOLUCIONES CON PYTHON\UNIDAD 4\Tarea4_Modulo_Embeddings\myvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

Respuesta: Spiderman
Score: 0.9504584074020386


AHORA HAREMOS VARIAS PRUEBAS CON EL PIPELINE DE RESPUESTA

In [2]:
examples = [
    {
        'question' : '¿Cual es la película favorita de Manuel Romero?',
        'context' : 'Manuel Romero está colaborando activamente con huggingface/transformers '
           'para traer el poder de las últimas técnicas de procesamiento de lenguaje natural al idioma español'
           'La película favorita de Manuel Romero es Spiderman'
    },
    {
        'question' : '¿Cual es la idea principal?',
        'context' : 'La novela narra el viaje del protagonista principal, Frodo Bolsón, hobbit de la Comarca, para destruir el Anillo Único y la consiguiente guerra que provocará el enemigo para recuperarlo, ya que es la principal fuente de poder de su creador, el señor oscuro Sauron. «Tres Anillos para los Reyes Elfos bajo el cielo'
    },
    {
        'question': '¿Qué personas han sido mencionadas?',
        'context' : 'La novela narra el viaje del protagonista principal, Frodo Bolsón, hobbit de la Comarca, para destruir el Anillo Único y la consiguiente guerra que provocará el enemigo para recuperarlo, ya que es la principal fuente de poder de su creador, el señor oscuro Sauron. «Tres Anillos para los Reyes Elfos bajo el cielo'
    },
    {
        'question': '¿Quien es el protragonista?',
        'context' : 'La novela narra el viaje del protagonista principal, Frodo Bolsón, hobbit de la Comarca, para destruir el Anillo Único y la consiguiente guerra que provocará el enemigo para recuperarlo, ya que es la principal fuente de poder de su creador, el señor oscuro Sauron. «Tres Anillos para los Reyes Elfos bajo el cielo'
    },
    {
        'question': '¿Cual es la principal fuente de poder?',
        'context' : 'La novela narra el viaje del protagonista principal, Frodo Bolsón, hobbit de la Comarca, para destruir el Anillo Único y la consiguiente guerra que provocará el enemigo para recuperarlo, ya que es la principal fuente de poder de su creador, el señor oscuro Sauron. «Tres Anillos para los Reyes Elfos bajo el cielo'
    },
    {
        'question': '¿Como se llama el libro?',
        'context' : 'La novela narra el viaje del protagonista principal, Frodo Bolsón, hobbit de la Comarca, para destruir el Anillo Único y la consiguiente guerra que provocará el enemigo para recuperarlo, ya que es la principal fuente de poder de su creador, el señor oscuro Sauron.' 
        'Nombre del libro: Tres Anillos para los Reyes Elfos bajo el cielo'
    },
    {
        'question': '¿Que sucedió con el estudiante de secundaria?',
        'context': 'Mordido por una araña genéticamente modificada, un estudiante de secundaria tímido y torpe obtiene increíbles capacidades como arácnido. Pronto comprenderá que su cometido es utilizarlas para luchar contra el mal y defender a sus vecinos.'
    },
    {
        'question': '¿Que obtiene el estudiante de secundaria?',
        'context' : 'Mordido por una araña genéticamente modificada, un estudiante de secundaria tímido y torpe obtiene increíbles capacidades como arácnido. Pronto comprenderá que su cometido es utilizarlas para luchar contra el mal y defender a sus vecinos.'
    }
    
]

def process_examples(examples):
    results=[]
    for example in examples:
        try:
            result = nlp(question=example['question'], context=example['context'])
            results.append(result)
            # Mostrar el resultado y el score
            print(f"Pregunta: {example['question']}")
            print(f"Respuesta: {result['answer']}")
            print(f"Score: {result['score']}\n")
        except KeyError as e:
            print(f"Error procesando el ejemplo: {example}")
            print(f"KeyError: {e}")
    return results


results = process_examples(examples)

Pregunta: ¿Cual es la película favorita de Manuel Romero?
Respuesta: Spiderman
Score: 0.9504584074020386

Pregunta: ¿Cual es la idea principal?
Respuesta: el Anillo Único
Score: 0.2904157042503357

Pregunta: ¿Qué personas han sido mencionadas?
Respuesta: Reyes Elfos
Score: 0.008702215738594532

Pregunta: ¿Quien es el protragonista?
Respuesta: Frodo Bolsón,
Score: 5.951296770945191e-06

Pregunta: ¿Cual es la principal fuente de poder?
Respuesta: señor oscuro Sauron.
Score: 0.17185266315937042

Pregunta: ¿Como se llama el libro?
Respuesta: Tres Anillos para los Reyes Elfos bajo el cielo
Score: 0.6884682178497314

Pregunta: ¿Que sucedió con el estudiante de secundaria?
Respuesta: Mordido por una araña genéticamente modificada,
Score: 0.467193067073822

Pregunta: ¿Que obtiene el estudiante de secundaria?
Respuesta: increíbles capacidades como arácnido.
Score: 0.8165550827980042



SE PUEDE UTILIZAR DISTILLBERT PARA REALIZAR MÁS TAREAS, PERO ESTE MODELO HA SIDO ENTRENADO PARA REALIZAR ESTA ACTIVIDAD EN ESPECÍFICO DE MEJOR MANERA.

PROBAREMOS CLASIFICAR TEXTO AHORA PARA VER SU EFICACIA. 

In [3]:
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax
import csv
import urllib.request
import tarfile

# Utilizamos un modelo BERT para el análisis de sentimiento
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es")
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es")

text = "Me encanta esta película"

# Tokenizamos el texto
encoded_input = tokenizer(text, return_tensors='pt', max_length=512, truncation=True)
        
        # Propagamos el input a través del modelo
raw_output = model(**encoded_input)

        # El output del modelo es una tupla, estamos interesados en el primer elemento de la tupla
        # que es un tensor de forma (1, num_labels). Tomamos el primer elemento de este tensor
        # y lo desacoplamos para obtener un array de numpy.
output_array = raw_output[0][0].detach().numpy()
        
        # Aplicamos la función softmax al array para obtener las probabilidades
probabilities = softmax(output_array)
        
outputs = probabilities

print(outputs)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[0.34069318 0.6593068 ]


PROBAREMOS AHORA MUCHOS TEXTOS NEGATIVOS Y POSITIVOS PARA VER SU RENDIMIENTO 

In [5]:
import os 
import random
from scipy.special import softmax
import numpy as np

np.set_printoptions(suppress=True)

#Ponemos la ruta de las reseñas postivas y negativas
pos_dir = 'aclImdb/test/pos/'
neg_dir = 'aclImdb/test/neg/'

#Obtenemos la lista de los archvos txt
pos_files = [os.path.join(pos_dir, f) for f in os.listdir(pos_dir) if f.endswith('.txt')]
neg_files = [os.path.join(neg_dir, f) for f in os.listdir(neg_dir) if f.endswith('.txt')]

#Elegimos 50 al azar
random_pos_files = random.sample(pos_files, 50)
random_neg_files = random.sample(neg_files, 50)

#Leemos los archivos
pos_texts = [open(file, 'r', encoding='utf-8').read() for file in random_pos_files]
neg_files = [open(file, 'r', encoding='utf-8').read() for file in random_neg_files]